# Slippage Simulator

In [1]:
from uniswappy import *
import time
import datetime
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt

### Setup 0x API

In [2]:
buy_token = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
sell_token = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
sell_amount = '10000000'

api = API0x(chain = Chain0x.ETHEREUM)
res = api.apply(sell_token, buy_token, sell_amount)

### Setup Uniswap Pool

In [3]:
prices = []

USER_NM = 'user0'
eth_amount = 1000
tkn_amount = eth_amount/float(res['price'])

eth = ERC20("WETH", None)
tkn = ERC20("USDC", None)
exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", address=None)

factory = UniswapFactory("ETH pool factory", None)
lp = factory.deploy(exchg_data)
lp.add_liquidity(USER_NM, eth_amount, tkn_amount, eth_amount, tkn_amount)
lp.summary()

Exchange WETH-USDC (LP)
Reserves: WETH = 1000, USDC = 3127941.9517103913
Liquidity: 55928.00686338099 



### Perform Mock Swap / Arbitrage

In [4]:
N = 20
tkn_price_arr = []
tstamp = []
MAX_SLEEP = 3

lp_tot_arr = []
x_amt_arr = []
y_amt_arr = []
lp_price_arr = []

swap_tstamp = []
swap_amts = []

market_tstamp = []
market_price = []
res = api.apply(sell_token, buy_token, sell_amount)
p = 1/float(res['price'])
correction = CorrectReserves(lp, x0 = p)

tdModel = TokenDeltaModel(30)

for k in range(N):
    
    remaining_sleep = MAX_SLEEP
    
    # RANDOM SWAP
    pause = random.uniform(0, remaining_sleep)
    time.sleep(pause) 
    rnd_amt = tdModel.delta()
    select_tkn = EventSelectionModel().bi_select(0.5)
    
    if(select_tkn == 0):
        out = Swap().apply(lp, eth, USER_NM, rnd_amt)  
    else:
        out = Swap().apply(lp, tkn, USER_NM, 0.5*p*rnd_amt) 
        
    swap_tstamp.append(datetime.datetime.now())
    swap_amts.append(rnd_amt)
    
    lp_price_arr.append(lp.get_price(correction.get_x_tkn()))  
    x_amt_arr.append(lp.get_reserve(correction.get_y_tkn()))  
    y_amt_arr.append(lp.get_reserve(correction.get_x_tkn()))                  
    tstamp.append(datetime.datetime.now())  
    remaining_sleep -= pause
    print(f"time-trade {datetime.datetime.now()}")    
    
    # MARKET ARBITRAGE
    pause = random.uniform(0, remaining_sleep)
    time.sleep(pause)     
    res = api.apply(sell_token, buy_token, sell_amount)
    p = 1/float(res['price'])
    
    market_tstamp.append(datetime.datetime.now())
    market_price.append(p)
    correction.apply(p)
                
    lp_price_arr.append(lp.get_price(correction.get_x_tkn()))  
    x_amt_arr.append(lp.get_reserve(correction.get_y_tkn()))  
    y_amt_arr.append(lp.get_reserve(correction.get_x_tkn()))                  
    tstamp.append(datetime.datetime.now())
    remaining_sleep -= pause
    print(f"time-arbitrage {datetime.datetime.now()}")           
                
    time.sleep(remaining_sleep) 


time-trade 2024-02-25 18:19:24.371585
time-arbitrage 2024-02-25 18:19:25.844237
time-trade 2024-02-25 18:19:28.511219
time-arbitrage 2024-02-25 18:19:29.522452
time-trade 2024-02-25 18:19:32.678175
time-arbitrage 2024-02-25 18:19:33.276581
time-trade 2024-02-25 18:19:33.857378
time-arbitrage 2024-02-25 18:19:35.509768
time-trade 2024-02-25 18:19:39.818372


KeyError: 'price'

### Plot Uniswap Liquidity Pool States

In [ ]:
fig, (p_ax, tkn1_ax, tkn2_ax, s_ax) = plt.subplots(nrows=4, sharex=False, sharey=False, figsize=(15, 15))
p_ax.plot(tstamp, lp_price_arr, label='pool')
p_ax.plot(market_tstamp, market_price, 'ro--', label='market')
p_ax.set_ylabel(f'{eth.token_name} / {tkn.token_name}', fontsize=16)
p_ax.legend(fontsize=16, facecolor="lightgray", loc='lower right')

tkn1_ax.plot(tstamp, x_amt_arr)
tkn1_ax.set_ylabel(f'{tkn.token_name} Reserves', fontsize=16)

tkn2_ax.plot(tstamp, y_amt_arr)
tkn2_ax.set_ylabel(f'{eth.token_name} Reserves', fontsize=16)

s_ax.plot(swap_tstamp, swap_amts)
s_ax.set_ylabel(f'Swap Amounts ({eth.token_name})', fontsize=16)

### Plot Swap Model

In [ ]:
r_vals = [tdModel.delta() for k in range(1000)] 
dens = sm.nonparametric.KDEUnivariate(r_vals)
dens.fit()

x = np.linspace(0,80,100)
y = dens.evaluate(x)

#rsamp = tdModel.delta()
rind = random.randint(0, len(dens.icdf))
rsamp = dens.icdf[rind]

residuals = np.abs(x - rsamp)
ind = np.argmin(residuals, axis=0) 

plt.hist(r_vals, density=True, bins=40, color='skyblue')
plt.plot(x, y, 'r')
plt.plot([rsamp,rsamp], [0,y[ind]], 'r', linewidth=2)
plt.plot(rsamp,y[ind],'ro') 
plt.title(f'Swap Model') 
plt.xlabel(f'{eth.token_name} Swap Amount') 